In [ ]:

import os
os.environ['KAGGLE_USERNAME'] = "bhavnoorsingh123" # username from the json file
os.environ['KAGGLE_KEY'] = "272115669b2aea31e435de4260678cd9" # key from the json file
!kaggle competitions download -c hms-harmful-brain-activity-classification

100% 18.4G/18.4G [03:26<00:00, 135MB/s]
100% 18.4G/18.4G [03:26<00:00, 95.6MB/s]


In [ ]:
!unzip /content/hms-harmful-brain-activity-classification.zip

Streaming output truncated to the last 5000 lines.
  inflating: train_spectrograms/2080462125.parquet  
  inflating: train_spectrograms/208059969.parquet  
  inflating: train_spectrograms/2080686415.parquet  
  inflating: train_spectrograms/2080973847.parquet  
  inflating: train_spectrograms/2081117549.parquet  
  inflating: train_spectrograms/2081307623.parquet  
  inflating: train_spectrograms/2081826095.parquet  
  inflating: train_spectrograms/2081944512.parquet  
  inflating: train_spectrograms/2082016410.parquet  
  inflating: train_spectrograms/2082370788.parquet  
  inflating: train_spectrograms/2082376822.parquet  
  inflating: train_spectrograms/2082907673.parquet  
  inflating: train_spectrograms/2083007837.parquet  
  inflating: train_spectrograms/2083142969.parquet  
  inflating: train_spectrograms/2083155730.parquet  
  inflating: train_spectrograms/208321640.parquet  
  inflating: train_spectrograms/2083642063.parquet  
  inflating: train_spectrograms/2084695153.parquet

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
from scipy.signal import butter, filtfilt
def butter_lowpass_filter(data, order):
    normal_cutoff = 30
    # Get the filter coefficients
    b, a = butter(order, normal_cutoff, btype='low', analog=False, fs=200)
    y = filtfilt(b, a, data)
    return y
df = pd.read_csv("train.csv")
df.head()

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


In [ ]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 40.8 MB/s eta 0:00:00


In [ ]:
from scipy import stats
def mean(x):
  return np.mean(x)

def std(x):
  return np.std(x)
def ptp(x):
  return np.ptp(x)
def var(x):
  return np.var(x)
def minim(x):
  return np.min(x)
def maxim(x):
  return np.max(x)
def argminim(x):
  return np.argmin(x)
def argmaxim(x):
  return np.argmax(x)
def rms(x):
  return np.sqrt(np.mean(x**2))
def abs_diff(x):
  return np.abs(np.sum(np.diff(x)))
def kurtosis(x):
  return stats.kurtosis(x)
def skewness(x):
  return stats.skew(x)

def returnAll(x):
  return np.array([mean(x), std(x), ptp(x), var(x), minim(x), maxim(x), argminim(x), argmaxim(x), rms(x), abs_diff(x), skewness(x), kurtosis(x)])

In [ ]:
#yet to do concatenate
import numpy as np
import cv2
import pywt
X_train_cwt = []
X_train_tdf = []
Y_train = []

fs=200
frequencies=np.array([5,10,15,20,25,30])/fs
scales=pywt.frequency2scale('morl',frequencies)
idx = 0
for j in range(len(df)):
  name = df.iloc[j]['eeg_id']
  offset = int(df.iloc[j]['eeg_label_offset_seconds'])
  myfile = pq.ParquetFile("/content/train_eegs/"+str(name)+".parquet")
  data = myfile.read().to_pandas().iloc[offset*200:(offset+50)*200]
  columns = data.columns
  base = []
  # for i in columns:
  #   coefs, freq = pywt.cwt(np.array(data[i]), np.array(range(1,201)), 'morl')
  #   scales = np.array(range(1,201))
  #   removedNoise = coefs[50:]
  #   base.append(removedNoise)
  for i in columns:
    base.append(pywt.cwt(butter_lowpass_filter(data[i], 2),scales,'morl')[0])
  # print(base[0].shape)
  base = np.concatenate(base)
  base=cv2.resize(base,(224,224))
  X_train_cwt.append(base)
  base = []
  for i in columns:
    base.append(returnAll(data[i]))
  base = np.concatenate(base)
  X_train_tdf.append(base)
  Y_train.append(df.iloc[j]['expert_consensus'])
  # dataframe.append(["img"+str(idx)+".jpg", df.iloc[j]['expert_consensus']])
  # cv2.imwrite("/content/images/"+"img"+str(idx)+".jpg", abs(base))
  # break
  idx+=1
  print(j)
  if j == 5000:
    break

Streaming output truncated to the last 5000 lines.
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
2

In [ ]:
# import numpy as np
# from sklearn.preprocessing import OneHotEncoder
# a = np.array(['seizure', 'lpd', 'lpd','lrda', 'grda'])
# enc=OneHotEncoder(sparse=False)

# a = enc.fit_transform(a.reshape(-1,1))
# print(a)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(sparse=False)
Y_train_enc = enc.fit_transform(np.array(Y_train).reshape(-1,1))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
from sklearn.model_selection import train_test_split
X_train_CNN, X_test_CNN, Y_train, Y_test = train_test_split(X_train_cwt, Y_train_enc, random_state=101)
# X_train_tdf, X_test_tdf, Y_train, Y_test = train_test_split(X_train_tdf, Y_train_enc, random_state=101)


In [ ]:
X_train_CNN = np.array(X_train_CNN)
Y_train=np.array(Y_train)

In [ ]:
X_train_CNN = X_train_CNN.reshape(-1,224,224,1)

In [ ]:
X_train_CNN.shape

(3750, 224, 224, 1)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Rescaling

model = Sequential()
# model.add(Rescaling(1./255))
model.add(Conv2D(64, (3,3), input_shape=(224,224,1), activation = 'relu'))
model.add(Conv2D(32, (3,3), activation = 'relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3,3), activation = 'relu'))
model.add(MaxPooling2D((2,2)))
# model.add(Conv2D(32, (3,3), activation = 'relu'))
# model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(6, activation = 'softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer = 'Adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train_CNN, Y_train, epochs = 20, validation_split = 0.2)

Epoch 1/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 16s 127ms/step - accuracy: 0.1576 - loss: nan - val_accuracy: 0.1280 - val_loss: nan
Epoch 2/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 16s 93ms/step - accuracy: 0.1611 - loss: nan - val_accuracy: 0.1280 - val_loss: nan
Epoch 3/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 93ms/step - accuracy: 0.1446 - loss: nan - val_accuracy: 0.1280 - val_loss: nan
Epoch 4/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 96ms/step - accuracy: 0.1419 - loss: nan - val_accuracy: 0.1280 - val_loss: nan
Epoch 5/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 92ms/step - accuracy: 0.1522 - loss: nan - val_accuracy: 0.1280 - val_loss: nan
Epoch 6/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 92ms/step - accuracy: 0.1552 - loss: nan - val_accuracy: 0.1280 - val_loss: nan
Epoch 7/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 91ms/step - accuracy: 0.1516 - loss: nan - val_accuracy: 0.1280 - val_loss: nan
Epoch 8/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 9s 93ms/step - accuracy: 0.1421 - loss: nan - val_accuracy: 0.1280 - val_loss: nan
Epoch 9/20
94/94 ━━━━━━━

In [ ]:
X_test_CNN = np.array(X_test_CNN)
X_test_CNN = X_test_CNN.reshape(-1, 224, 224, 1)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(np.argmax(Y_test, axis = 1), np.argmax(model.predict(X_test_CNN), axis = 1))*100, "%", sep = "")

40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step
23.181454836131095%
